In [13]:
import pandas as pd
import numpy as np
import gc
import re
import MeCab

In [14]:
work_path_lizhi = '...'
y_cate_path = 'paring_table.csv.gz'
df_ycate = pd.read_csv(work_path_lizhi + y_cate_path, compression='gzip')

In [18]:
len(df_ycate.id.value_counts())

181838

In [2]:
account_path = '...'
order_path = '...'
deal_path = '...'
deal_profile_path = '...'
work_path = '...'
deal_profile_path2 = '...'
df_account = pd.read_csv(work_path + account_path, sep = '\t', compression='gzip')
df_order = pd.read_csv(work_path + order_path, sep = '\t', compression='gzip')
df_deal = pd.read_csv(work_path + deal_path, sep = '\t', compression='gzip')
df_dealprofile = pd.read_csv(work_path + deal_profile_path, sep = '\t', compression='gzip')
df_dealprofile2 = pd.read_csv(work_path + deal_profile_path2, sep = '\t', compression='gzip')

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,14,15,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,9,10,13,17,18,24,25,26,29

In [2]:
df_order.ins_tm.min()

'2016-10-01 00:00:01'

In [6]:
#delete website tag 
pattern = re.compile(r'<(.*?)>')
df_dealprofile.deal_description = df_dealprofile.deal_description.map(lambda x: str(re.sub(pattern, '', str(x))))
df_dealprofile2.deal_description = df_dealprofile2.deal_description.map(lambda x: str(re.sub(pattern, '', str(x))))

In [7]:
df_dealprofile.deal_title = df_dealprofile.deal_title.map(lambda x: str(re.sub(pattern, '', str(x))))
df_dealprofile2.deal_title = df_dealprofile2.deal_title.map(lambda x: str(re.sub(pattern, '', str(x))))

In [8]:
# preprocess deal_description
mecab = MeCab.Tagger('-Ochasen')
def rawdcrp2tokendcrp(rawdescription):
    deal_des = []
    for line in rawdescription:
        tokenlst = mecab.parse(line)
        pattern1 = re.compile(r'(.*?)(名詞|動詞|形容詞)(.*?)')
        new_dcp1 = pattern1.findall(tokenlst)
        pattern2 = re.compile('(.*?)\t')
        sentence_des = []
        for token in range(len(new_dcp1)):
            matchobjdes = re.match(pattern2, new_dcp1[token][0])
            if(matchobjdes != None):
                sentence_des.append(matchobjdes.group(1))
        deal_des.append(sentence_des)
    return deal_des
df_dealprofile.deal_description = df_dealprofile.deal_description.map(lambda x: x.split('。'))
df_dealprofile.deal_description = df_dealprofile.deal_description.map(lambda x: rawdcrp2tokendcrp(x))
df_dealprofile2.deal_description = df_dealprofile2.deal_description.map(lambda x: x.split('。'))
df_dealprofile2.deal_description = df_dealprofile2.deal_description.map(lambda x: rawdcrp2tokendcrp(x)) 

In [9]:
df_dealprofile.deal_title = df_dealprofile.deal_title.map(lambda x: x.split('。'))
df_dealprofile.deal_title = df_dealprofile.deal_title.map(lambda x: rawdcrp2tokendcrp(x)) 
df_dealprofile2.deal_title = df_dealprofile2.deal_title.map(lambda x: x.split('。'))
df_dealprofile2.deal_title = df_dealprofile2.deal_title.map(lambda x: rawdcrp2tokendcrp(x)) 

In [10]:
df_dealprofile.to_csv('df_dealprofile_afterprep201804.csv.gz', compression = 'gzip')
df_dealprofile2.to_csv('df_dealprofile_afterprep201712.csv.gz', compression = 'gzip')

In [11]:
import datetime
df_dealprofile['ins_tm'] = pd.to_datetime(df_dealprofile['ins_tm'],format='%Y-%m-%d %H:%M:%S')
df_dealprofile2['ins_tm'] = pd.to_datetime(df_dealprofile2['ins_tm'],format='%Y-%m-%d %H:%M:%S')
df_dealprofile['date'] = df_dealprofile.ins_tm.dt.date
df_dealprofile2['date'] = df_dealprofile2.ins_tm.dt.date

In [12]:
train_dealprofile = df_dealprofile.loc[df_dealprofile.date <= datetime.date(2017,9,14)]
train_dealprofile2 = df_dealprofile2.loc[df_dealprofile2.date <= datetime.date(2017,9,14)]

In [ ]:
work_path2 = '...'
url_path0 = '...'
url_path1 = '...'
url_path2 = '...'
df_url0 = pd.read_csv(work_path2 + url_path0, sep = ',', compression='gzip')
df_url1 = pd.read_csv(work_path2 + url_path1, sep = ',', compression='gzip')
df_url2 = pd.read_csv(work_path2 + url_path2, sep = ',', compression='gzip')
df_url = df_url0.append(df_url1)
df_url = df_url.append(df_url2)
(df_url.description.notnull() & df_url.keywords.notnull()).sum()
(df_url.description.notnull() & df_url.title.notnull()).sum()
(df_url.description.notnull() & df_url.title.notnull() & df_url.keywords.notnull()).sum()
df_url.notnull().sum()
df_url = df_url[['index', 'description', 'title']]
df_url = df_url.dropna()
df_url.shape
df_url.description = df_url.description.map(lambda x: x.split('。'))
df_url.description = df_url.description.map(lambda x: rawdcrp2tokendcrp(x))
df_url.title = df_url.title.map(lambda x: x.split('。'))
df_url.title = df_url.title.map(lambda x: rawdcrp2tokendcrp(x))

for line in df_url.description.values:
    corpus = corpus + line
for line in df_url.title.values:
    corpus = corpus + line
corpus = []
np.save('corpus.npy', corpus)

In [13]:
corpus = []
for line in train_dealprofile.deal_description.values:
    corpus.extend(line)
for line in train_dealprofile.deal_title.values:
    corpus.extend(line)
for line in train_dealprofile2.deal_description.values:
    corpus.extend(line)
for line in train_dealprofile2.deal_title.values:
    corpus.extend(line)

In [14]:
np.save('corpus_dealonly.npy', corpus)

In [4]:
import pandas as pd
import numpy as np
import gc
import re
import MeCab
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

In [5]:
test_cor = np.load('corpus.npy')

In [17]:
import math

In [18]:
last_count = 0
new_count = 0 
for i in range(6):
    new_count = ((df_order.deal_id.value_counts() <= (10* math.pow(10, i))) == True).sum() - last_count
    print(new_count)
    last_count += new_count 

47255
19198
1860
136
9
0


In [2]:
sell_count = df_order.deal_id.value_counts()
sell = pd.DataFrame(sell_count)
sell = sell.rename(columns= {'deal_id':'sell_count'})
sell = sell.reset_index()
sell.columns = ['deal_id','sell_count']

In [3]:
df_dealprofile.shape

(73965, 35)

In [4]:
result = pd.merge(df_dealprofile, sell, how ='inner', on= ['deal_id'])

In [5]:
result.shape

(2162, 36)

In [6]:
result = pd.merge(df_dealprofile, df_order, how ='inner', on= ['deal_id'])

In [15]:
result = pd.merge(df_dealprofile, df_deal, how ='inner', on= ['deal_id'])

In [16]:
result = pd.merge(df_dealprofile2, sell, how ='inner', on= ['deal_id'])

In [19]:
result = pd.merge(df_dealprofile, df_deal, how ='inner', on= ['deal_id'])

In [25]:
result = pd.merge(result, sell, how = 'inner', on=['deal_id'])

In [26]:
result = result[['deal_id','sell_count','deal_category_cd',]]

In [36]:
df_new = pd.read_csv('df_new.csv.gz', sep = ',', compression='gzip')

In [48]:
df_new.deal_description = df_new.deal_description.map(lambda x: eval(x))

In [58]:
result = pd.merge(sell, df_new, how = 'inner', on=['deal_id'])

# session-based embedding

In [96]:
df_order.deal_id = df_order.deal_id.map(lambda x: str(x))

In [98]:
grouped_order = df_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))

In [100]:
Input_Data = pd.DataFrame(grouped_order)
Input_Data = Input_Data.reset_index(drop = True)
Input_Data = Input_Data.iloc[:,0].values
corpus = []
for line in Input_Data:
    words = [x for x in line.split(' ')]
    corpus.append(words)

In [102]:
np.save('session_based_corpus.npy', corpus)

In [104]:
grouped_order = df_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))
Input_Data = pd.DataFrame(grouped_order)

In [17]:
df_order = df_order[['account_id', 'deal_id', 'ins_tm']]

In [18]:
offset = np.zeros(df_order.account_id.nunique()+1, dtype = np.int32)

In [20]:
offset.shape

(341410,)

In [22]:
offset[1:] = df_order.groupby('account_id').size().cumsum()

In [24]:
item_id = df_order.deal_id.unique()

In [25]:
item_id.shape

(68458,)

In [27]:
item2idx = pd.Series(data=np.arange(len(item_id)),index=item_id)

itemmap = pd.DataFrame({'item_ID':item_id,'item_idx':item2idx[item_id].values})
        

In [2]:
df_order = df_order[['account_id', 'deal_id']]
df_order['rating'] = 1

In [3]:
df_order.account_id = df_order.account_id.astype('category').cat.codes.values
df_order.deal_id = df_order.deal_id.astype('category').cat.codes.values

In [4]:
import random

In [5]:
item_list = list(df_order.deal_id.unique())
bought_list = list(df_order[df_order.account_id == 24350].deal_id.values)
negative_list = list(set(item_list)^set(bought_list))

In [6]:
len(item_list), len(bought_list), len(negative_list)

(68458, 23, 68435)

In [15]:
Negative_sample = []
for i in range(len(bought_list)):
        some = random.sample(negative_list, 1)[0]
        Negative_sample.append([24350, some, 0])

In [13]:
df_order = df_order[['account_id', 'deal_id']]

In [18]:
df_order['value_count'] = 1

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_buy = df_order.groupby(['account_id', 'deal_id']).sum()